<a href="https://colab.research.google.com/github/pabrey/datasciencecoursera/blob/master/4_a_file_analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previous

In [1]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
import json
from google.colab import files
import time
from datetime import datetime
import math

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create routes data

## create routes information

In [2]:
Location = 'drive/My Drive/data/prosae/original-log.csv'
Separator = ';'
df =  pd.read_csv(Location, sep=Separator)
df.columns = ["CASE_ID", "ACTIVITY",	"DATE",	"INDEX",	"COLOR"]
df_order = df.sort_values(['CASE_ID', 'INDEX'])
df_order.head()

,CASE_ID,ACTIVITY,DATE,INDEX,COLOR
0,173000001,ADMISION,01/01/2017 0:06,0,NaN
3,173000001,TRIAJE,01/01/2017 0:11,1,amarillo
16,173000001,ALTA,01/01/2017 0:58,2,NaN
1,173000002,ADMISION,01/01/2017 0:07,0,NaN
4,173000002,TRIAJE,01/01/2017 0:13,1,verde


In [4]:

# creamos  las columnas
# join function => The join() method takes all items in an iterable and joins them into one string.
# reset _index -> Pandas reset_index() is a method to reset index of a Data Frame. reset_index() method sets a list of integer ranging from 0 to length of data as index.
df_routes = df_order.groupby('CASE_ID')['ACTIVITY'].agg(lambda x: ','.join(x)).reset_index()
df_routes['SIZE'] = df_routes['ACTIVITY'].apply(lambda x: len(x.split(',')))

df_routes.columns = ['CASE_ID', 'PATH', 'SIZE_PATH']
df_routes['USER_PER_SIZE_PATH'] = df_routes.groupby('SIZE_PATH')["CASE_ID"].transform(len)
df_routes['USER_PER_PATH'] = df_routes.groupby('PATH')["CASE_ID"].transform(len)
df_routes.reset_index()


print("min size path: "+str(min(df_routes["SIZE_PATH"])))
print("Max size path: "+str(max(df_routes["SIZE_PATH"])))
print("Mean size path: "+str(df_routes["SIZE_PATH"].mean()))
print("MEdian size path: "+str(df_routes["SIZE_PATH"].median()))
print("Num users per route mean: "+str(df_routes["USER_PER_SIZE_PATH"].mean()))
print("Num users per route median: "+str(df_routes["USER_PER_SIZE_PATH"].median()))
number_user = df_routes["USER_PER_SIZE_PATH"].drop_duplicates().sum()
print("Num users : " + str(number_user))
df_routes["PERCENTAGE_PER_SIZE"] = df_routes["USER_PER_SIZE_PATH"]/number_user
df_routes["PERCENTAGE_PER_PATH"] = df_routes["USER_PER_PATH"]/number_user
df_routes[["SIZE_PATH", "USER_PER_SIZE_PATH", "PERCENTAGE_PER_SIZE", "PATH", "USER_PER_PATH", "PERCENTAGE_PER_PATH"]].sort_values("SIZE_PATH").drop_duplicates().head()

min size path: 3
Max size path: 17
Mean size path: 4.42453417222664
MEdian size path: 5.0
Num users per route mean: 42443.59784975974
Num users per route median: 47810.0
Num users : 115708


,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,PATH,USER_PER_PATH,PERCENTAGE_PER_PATH
0,3,50387,0.435467,"ADMISION,TRIAJE,ALTA",50387,0.435467
37082,4,1011,0.008738,"ADMISION,TRIAJE,RX SOLICITUD,ALTA",841,0.007268
71412,4,1011,0.008738,"ADMISION,TRIAJE,INTERCONSULTA,ALTA",170,0.001469
77339,5,47810,0.413195,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",47530,0.410775
77171,5,47810,0.413195,"ADMISION,TRIAJE,INTERCONSULTA,RESPUESTA,ALTA",273,0.002359


In [5]:
#check safety, sum must be 1
print(sum(df_routes.groupby("SIZE_PATH")["PERCENTAGE_PER_SIZE"].min()))
print(sum(df_routes.groupby("PATH")["PERCENTAGE_PER_PATH"].min()))

1.0
1.00000000000001


In [6]:
df_join = df.merge(df_routes).sort_values(["CASE_ID", "DATE"])
df_join =  df_join[["CASE_ID", "ACTIVITY", "DATE", "INDEX", "PATH", "SIZE_PATH", "USER_PER_SIZE_PATH", "PERCENTAGE_PER_SIZE",  "USER_PER_PATH", "PERCENTAGE_PER_PATH"]]
df_join = df_join.sort_values(['CASE_ID', 'INDEX'])
df_join.head()

,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH
0,173000001,ADMISION,01/01/2017 0:06,0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467
1,173000001,TRIAJE,01/01/2017 0:11,1,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467
2,173000001,ALTA,01/01/2017 0:58,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467
3,173000002,ADMISION,01/01/2017 0:07,0,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775
4,173000002,TRIAJE,01/01/2017 0:13,1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775


## process times

In [7]:
df_timestamp = df_join.copy()
#filter just the first id
#df_timestamp['TIMESTAMP'] = (pd.DatetimeIndex(df_timestamp.DATE).asi8)/10000000000
df_timestamp["TIMESTAMP"] = (df_timestamp['DATE']).apply(lambda x:  time.mktime( datetime.strptime(str(x), '%d/%m/%Y %H:%M').timetuple()))
 
df_timestamp["ArrivalTime"] = df_timestamp["TIMESTAMP"] 
df_timestamp["ServiceStartTime"] =  df_timestamp["TIMESTAMP"] 
df_timestamp["DepartureTime"] = 0

#normalize
min_value =  df_timestamp["ArrivalTime"].min()-1
df_timestamp["ArrivalTime"] = df_timestamp["ArrivalTime"]-min_value
df_timestamp["ServiceStartTime"] = df_timestamp["ServiceStartTime"]-min_value
df_timestamp.head()

,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,ArrivalTime,ServiceStartTime,DepartureTime
0,173000001,ADMISION,01/01/2017 0:06,0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,1.0,1.0,0
1,173000001,TRIAJE,01/01/2017 0:11,1,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,301.0,301.0,0
2,173000001,ALTA,01/01/2017 0:58,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483232e+09,3121.0,3121.0,0
3,173000002,ADMISION,01/01/2017 0:07,0,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483229e+09,61.0,61.0,0
4,173000002,TRIAJE,01/01/2017 0:13,1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483230e+09,421.0,421.0,0


establish departure time

In [8]:
data_time = df_timestamp.sort_values(["CASE_ID", "INDEX"]).reset_index().copy()


def processDepartureTime(nodes):

  temp_time = 0
  len_nodes = len(nodes)
  sal = []
  mi_index = 0
  for index, node in nodes.iteritems():
    mi_index = mi_index +1
  
    if (mi_index == len_nodes):
      sal.append(node)
    else:
      sal.append((nodes[index+1]))

  return sal
      

data_time["DepartureTime"] = data_time.groupby("CASE_ID")["ServiceStartTime"].transform(lambda x: processDepartureTime(x))
data_time['Difference'] = data_time['DepartureTime'] - data_time['ServiceStartTime']
data_time.head()


,index,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,ArrivalTime,ServiceStartTime,DepartureTime,Difference
0,0,173000001,ADMISION,01/01/2017 0:06,0,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,1.0,1.0,301.0,300.0
1,1,173000001,TRIAJE,01/01/2017 0:11,1,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483229e+09,301.0,301.0,3121.0,2820.0
2,2,173000001,ALTA,01/01/2017 0:58,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.483232e+09,3121.0,3121.0,3121.0,0.0
3,3,173000002,ADMISION,01/01/2017 0:07,0,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483229e+09,61.0,61.0,421.0,360.0
4,4,173000002,TRIAJE,01/01/2017 0:13,1,"ADMISION,TRIAJE,RX SOLICITUD,RX EXPLORACION,ALTA",5,47810,0.413195,47530,0.410775,1.483230e+09,421.0,421.0,1081.0,660.0


In [9]:
data_time["USER_PER_SIZE_PATH"].unique()

array([50387, 47810,   761,  7557,  4450,  2746,  1011,   537,    54,
          32,   255,    89,    13,     4,     2])

safety check, check the data, al difference is > 0

In [10]:
id_neg = data_time[data_time["Difference"] < 0]["CASE_ID"]
len(id_neg)

0

# search for inconstancies

In [12]:
#alta difference always must be 0
df_filter = data_time.copy().sort_values(["CASE_ID", "INDEX"])
df_filter[(df_filter["ACTIVITY"] == "ALTA") & (df_filter["Difference"]>0)]

,index,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,ArrivalTime,ServiceStartTime,DepartureTime,Difference


In [13]:
#last element must be ALTA
df_filter.tail(1)

,index,CASE_ID,ACTIVITY,DATE,INDEX,PATH,SIZE_PATH,USER_PER_SIZE_PATH,PERCENTAGE_PER_SIZE,USER_PER_PATH,PERCENTAGE_PER_PATH,TIMESTAMP,ArrivalTime,ServiceStartTime,DepartureTime,Difference
511953,511953,173181031,ALTA,01/01/2018 2:35,2,"ADMISION,TRIAJE,ALTA",3,50387,0.435467,50387,0.435467,1.514774e+09,31544941.0,31544941.0,31544941.0,0.0


all CASE_ID must have a ADMISION and a ALTA

In [14]:
df_general = df_filter["CASE_ID"].unique()
df_ALTA = df_filter[df_filter["ACTIVITY"] == "ALTA"]["CASE_ID"].unique()
set(df_general) - set(df_ALTA)

set()

In [15]:
df_general = df_filter["CASE_ID"].unique()
df_ALTA = df_filter[df_filter["ACTIVITY"] == "ADMISION"]["CASE_ID"].unique()
set(df_general) - set(df_ALTA)

set()